In [1]:
from transformers import pipeline, BertForSequenceClassification, DistilBertForSequenceClassification, RobertaForSequenceClassification, XLNetForSequenceClassification,BertTokenizer, DistilBertTokenizer, RobertaTokenizer, XLNetTokenizer
import alpaca_trade_api as tradeapi
import pandas as pd
import matplotlib.pyplot as plt
import torch
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_val_score, StratifiedKFold
import numpy as np
from multiprocessing import Pool
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
import xgboost as xgb
import tensorflow as tf
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, GRU, Dropout
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler
import torch
from transformers import BertModel, BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber, LogCosh
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_log_error, r2_score, accuracy_score


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
data = pd.read_excel("BSE500 as of Jul 24 20231_gyluxqnl.xlsx")
data.head(2)

,Ticker,Name,Price,Bloomberg Close Price,Comparison,Volume:Y-5,Volume:Y-4,Volume:Y-3,Volume:Y-2,Volume:Y-1,...,High Px:Y-4,High Px:Y-3,High Px:Y-2,High Px:Y-1,High Px:D-1,Open Px:Y-5,Open Px:Y-4,Open Px:Y-3,Open Px:Y-2,Open Px:Y-1
0,LLOYDSME IB Equity,Lloyds Metals & Energy Ltd,576.05,576.05,True,40477.0,11641.0,127467.0,557635.0,125063.0,...,8.8,11.55,52.900002,147.649994,580.000000,13.65,8.71,11.1,52.900002,146.899994
1,MAZDOCKS IB Equity,Mazagon Dock Shipbuilders Ltd,1955.45,1955.45,True,0.0,0.0,0.0,1239257.0,216111.0,...,0.0,0.00,274.100006,277.850006,1957.800049,0.00,0.00,0.0,274.100006,272.700012


In [4]:
features = ['Price', 'Volume:Y-5', 'Volume:Y-4', 'Volume:Y-3', 'Volume:Y-2', 'Volume:Y-1', 
            'Total Return:Y-4', 'Total Return:Y-3', 'Total Return:Y-2', 
            'Total Return:Y-1', 'Total Return:Y-5',  
            'Low Px:Y-5', 'Low Px:Y-4', 'Low Px:Y-3', 'Low Px:Y-2', 'Low Px:Y-1', 
            'High Px:Y-5', 'High Px:Y-4', 'High Px:Y-3', 'High Px:Y-2', 'High Px:Y-1', 
            'Bloomberg Close Price', 'Open Px:Y-5', 'Open Px:Y-4', 'Open Px:Y-3', 'Open Px:Y-2', 'Open Px:Y-1']

data1 = data[features]

# Handle missing values (example: fill with median)
data1.fillna(data1.median(), inplace=True)

C:\Users\goeld\AppData\Local\Temp\ipykernel_44996\3552615449.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.fillna(data1.median(), inplace=True)


In [5]:
data1.columns

Index(['Price', 'Volume:Y-5', 'Volume:Y-4', 'Volume:Y-3', 'Volume:Y-2',
       'Volume:Y-1', 'Total Return:Y-4', 'Total Return:Y-3',
       'Total Return:Y-2', 'Total Return:Y-1', 'Total Return:Y-5',
       'Low Px:Y-5', 'Low Px:Y-4', 'Low Px:Y-3', 'Low Px:Y-2', 'Low Px:Y-1',
       'High Px:Y-5', 'High Px:Y-4', 'High Px:Y-3', 'High Px:Y-2',
       'High Px:Y-1', 'Bloomberg Close Price', 'Open Px:Y-5', 'Open Px:Y-4',
       'Open Px:Y-3', 'Open Px:Y-2', 'Open Px:Y-1'],
      dtype='object')

ANOVA TEST

In [6]:
data = data[['Volume:Y-5', 'Volume:Y-4', 'Volume:Y-3', 'Volume:Y-2', 'Volume:Y-1', 'Volume:D-1',
          'Total Return:Y-4', 'Total Return:Y-3', 'Total Return:Y-2', 'Total Return:Y-1', 'Total Return:Y-5', 'Total Return:D-1',
          'Low Px:Y-5', 'Low Px:Y-4', 'Low Px:Y-3', 'Low Px:Y-2', 'Low Px:Y-1', 'Low Px:D-1',
          'High Px:Y-5', 'High Px:Y-4', 'High Px:Y-3', 'High Px:Y-2', 'High Px:Y-1', 'High Px:D-1',
          'Open Px:Y-5', 'Open Px:Y-4', 'Open Px:Y-3', 'Open Px:Y-2', 'Open Px:Y-1', "Bloomberg Close Price"]]

In [8]:
# Define features and target
X = data[['Volume:Y-5', 'Volume:Y-4', 'Volume:Y-3', 'Volume:Y-2', 'Volume:Y-1',
          'Total Return:Y-4', 'Total Return:Y-3', 'Total Return:Y-2', 'Total Return:Y-1', 'Total Return:Y-5', 
          'Low Px:Y-5', 'Low Px:Y-4', 'Low Px:Y-3', 'Low Px:Y-2', 'Low Px:Y-1', 'Low Px:D-1',
          'High Px:Y-5', 'High Px:Y-4', 'High Px:Y-3', 'High Px:Y-2', 'High Px:Y-1', 
          'Open Px:Y-5', 'Open Px:Y-4', 'Open Px:Y-3', 'Open Px:Y-2', 'Open Px:Y-1']]

y = data['Bloomberg Close Price']

In [9]:
from scipy import stats

# ANOVA test for each feature
anova_results = {}
for col in X.columns:
    groups = [y[data[col] == val] for val in data[col].unique()]
    f_val, p_val = stats.f_oneway(*groups)
    anova_results[col] = {'F-Value': f_val, 'P-Value': p_val}

anova_df = pd.DataFrame(anova_results).T
print(anova_df)


                       F-Value       P-Value
Volume:Y-5        5.240413e+01  5.636527e-51
Volume:Y-4        7.775297e+01  3.786011e-49
Volume:Y-3        1.223694e+02  6.248265e-46
Volume:Y-2        2.254353e+02  2.213627e-29
Volume:Y-1        4.668531e+02  4.454512e-08
Total Return:Y-4  7.775297e+01  3.786011e-49
Total Return:Y-3  1.223694e+02  6.248265e-46
Total Return:Y-2  2.254353e+02  2.213627e-29
Total Return:Y-1  4.668531e+02  4.454512e-08
Total Return:Y-5  5.240413e+01  5.636527e-51
Low Px:Y-5        5.112348e+01  2.210939e-52
Low Px:Y-4        8.724872e+01  7.460375e-56
Low Px:Y-3        1.126601e+02  1.073386e-49
Low Px:Y-2        2.948690e+02  3.733241e-40
Low Px:Y-1        7.318693e+02  7.539734e-15
Low Px:D-1        5.110447e+07  3.788485e-12
High Px:Y-5       5.664958e+01  1.198025e-56
High Px:Y-4       7.849396e+01  6.695004e-55
High Px:Y-3       1.378993e+02  3.902411e-53
High Px:Y-2       2.002345e+02  3.347687e-34
High Px:Y-1       2.647955e+02  1.998726e-12
Open Px:Y-

Chi-Square Test

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import chi2

1. Discretize y (Bloomberg Close Price):

In [13]:
# Discretize y (Bloomberg Close Price) into quantiles
y_discrete = pd.qcut(y.rank(method='first'), q=5, labels=False)

# Chi-square test
chi_scores, p_values = chi2(X_discrete, y_discrete)
chi_square_df = pd.DataFrame({'Feature': X.columns, 'Chi2 Score': chi_scores, 'P-value': p_values})

print(chi_square_df)



             Feature  Chi2 Score        P-value
0         Volume:Y-5   54.227970   4.714854e-11
1         Volume:Y-4   52.816287   9.310405e-11
2         Volume:Y-3   76.044208   1.198176e-15
3         Volume:Y-2  136.113762   1.916368e-28
4         Volume:Y-1  133.108960   8.421911e-28
5   Total Return:Y-4   25.308911   4.360278e-05
6   Total Return:Y-3    5.023812   2.848630e-01
7   Total Return:Y-2    8.720495   6.847837e-02
8   Total Return:Y-1   10.824257   2.861176e-02
9   Total Return:Y-5   39.608020   5.216447e-08
10        Low Px:Y-5  187.624802   1.716552e-39
11        Low Px:Y-4  236.214158   6.062961e-50
12        Low Px:Y-3  279.833762   2.420102e-59
13        Low Px:Y-2  360.036584   1.193360e-76
14        Low Px:Y-1  437.713168   1.967581e-93
15        Low Px:D-1  488.555941  2.000249e-104
16       High Px:Y-5  182.272772   2.423269e-38
17       High Px:Y-4  238.358663   2.093648e-50
18       High Px:Y-3  279.833762   2.420102e-59
19       High Px:Y-2  359.406584   1.632

2. Ensure Non-Negative Values in Features:

In [14]:
# Ensure non-negative values in X_discrete
X_discrete = X.apply(lambda x: pd.qcut(x.rank(method='first'), q=5, labels=False))
X_discrete = X_discrete - X_discrete.min().min()  # Shift to make non-negative if necessary

# Chi-square test
chi_scores, p_values = chi2(X_discrete, y_discrete)
chi_square_df = pd.DataFrame({'Feature': X.columns, 'Chi2 Score': chi_scores, 'P-value': p_values})

print(chi_square_df)


             Feature  Chi2 Score        P-value
0         Volume:Y-5   54.227970   4.714854e-11
1         Volume:Y-4   52.816287   9.310405e-11
2         Volume:Y-3   76.044208   1.198176e-15
3         Volume:Y-2  136.113762   1.916368e-28
4         Volume:Y-1  133.108960   8.421911e-28
5   Total Return:Y-4   25.308911   4.360278e-05
6   Total Return:Y-3    5.023812   2.848630e-01
7   Total Return:Y-2    8.720495   6.847837e-02
8   Total Return:Y-1   10.824257   2.861176e-02
9   Total Return:Y-5   39.608020   5.216447e-08
10        Low Px:Y-5  187.624802   1.716552e-39
11        Low Px:Y-4  236.214158   6.062961e-50
12        Low Px:Y-3  279.833762   2.420102e-59
13        Low Px:Y-2  360.036584   1.193360e-76
14        Low Px:Y-1  437.713168   1.967581e-93
15        Low Px:D-1  488.555941  2.000249e-104
16       High Px:Y-5  182.272772   2.423269e-38
17       High Px:Y-4  238.358663   2.093648e-50
18       High Px:Y-3  279.833762   2.420102e-59
19       High Px:Y-2  359.406584   1.632

Data Preprocessing

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, GRU, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from tensorflow.keras.callbacks import ModelCheckpoint


# Load your dataset
data = pd.read_excel("BSE500 as of Jul 24 20231_gyluxqnl.xlsx")

# Define features and target
X = data[['Volume:Y-5', 'Volume:Y-4', 'Volume:Y-3', 'Volume:Y-2', 'Volume:Y-1', 
          'Total Return:Y-4', 'Total Return:Y-3', 'Total Return:Y-2', 'Total Return:Y-1', 'Total Return:Y-5', 
          'Low Px:Y-5', 'Low Px:Y-4', 'Low Px:Y-3', 'Low Px:Y-2', 'Low Px:Y-1', 
          'High Px:Y-5', 'High Px:Y-4', 'High Px:Y-3', 'High Px:Y-2', 'High Px:Y-1', 
          'Open Px:Y-5', 'Open Px:Y-4', 'Open Px:Y-3', 'Open Px:Y-2', 'Open Px:Y-1']]
y = data['Bloomberg Close Price']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


Model Definitions and Training

In [19]:
# Model Definitions

# Adjust ModelCheckpoint for LSTM, GRU, and RNN models
lstm_checkpoint = ModelCheckpoint('lstm_model.keras', save_best_only=True)
gru_checkpoint = ModelCheckpoint('gru_model.keras', save_best_only=True)
rnn_checkpoint = ModelCheckpoint('rnn_model.keras', save_best_only=True)

def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def create_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(50, return_sequences=True, input_shape=input_shape))
    model.add(GRU(50))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def create_rnn_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=input_shape))  # Simple RNN layer
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Linear Regression
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()

# Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# XGBoost Regressor
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

# SVM Regressor
svm_model = SVR()

# Define and Train Models
# Linear Regression
linear_model.fit(X_train, y_train)
y_pred_linear = linear_model.predict(X_test)

# Random Forest
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

# XGBoost
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

# LSTM
lstm_model = create_lstm_model((X_train.shape[1], 1))
lstm_model.fit(X_train.reshape(X_train.shape[0], X_train.shape[1], 1), y_train, epochs=10, batch_size=32, verbose=1, callbacks=[ModelCheckpoint('lstm_model.keras', save_best_only=True)])

# GRU
gru_model = create_gru_model((X_train.shape[1], 1))
gru_model.fit(X_train.reshape(X_train.shape[0], X_train.shape[1], 1), y_train, epochs=10, batch_size=32, verbose=1, callbacks=[ModelCheckpoint('gru_model.keras', save_best_only=True)])

# RNN
rnn_model = create_rnn_model((X_train.shape[1], 1))
rnn_model.fit(X_train.reshape(X_train.shape[0], X_train.shape[1], 1), y_train, epochs=10, batch_size=32, verbose=1, callbacks=[ModelCheckpoint('rnn_model.keras', save_best_only=True)])

# Predictions
y_pred_lstm = lstm_model.predict(X_test.reshape(X_test.shape[0], X_test.shape[1], 1))
y_pred_gru = gru_model.predict(X_test.reshape(X_test.shape[0], X_test.shape[1], 1))
y_pred_rnn = rnn_model.predict(X_test.reshape(X_test.shape[0], X_test.shape[1], 1))


c:\Users\goeld\anaconda3\envs\assistant\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 51679988.0000
Epoch 2/10
 9/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 58113416.0000

c:\Users\goeld\anaconda3\envs\assistant\lib\site-packages\keras\src\callbacks\model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 54358440.0000
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 38488744.0000
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 34565844.0000
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 22312138.0000
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 42275096.0000
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 40188492.0000
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 46583252.0000
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 22878738.0000
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 57010512.0000
Epoch 1/10


c:\Users\goeld\anaconda3\envs\assistant\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 27579928.0000
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 44385700.0000
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 30158380.0000
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 27880002.0000
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 59986296.0000
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 61114912.0000
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 56202080.0000
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 19835428.0000
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 50239132.0000
Epoch 1/10


c:\Users\goeld\anaconda3\envs\assistant\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 59770776.0000
Epoch 2/10
 9/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 31769346.0000

c:\Users\goeld\anaconda3\envs\assistant\lib\site-packages\keras\src\callbacks\model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 35736904.0000
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 22424914.0000
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 24173834.0000
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 36534048.0000
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 25609868.0000
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 74804952.0000
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 46481644.0000
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 37584748.0000
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 93370504.0000
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 111ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 140ms/step
1/4 ━━━━━━━━━━━━━━━━━━━━ 1s 362ms/stepWARNING:tensorflow:6 out of the last 12 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x0000025C6DF98C10> triggered tf.function retracing. Tracing is expensive and the excessive numbe

In [22]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def calculate_metrics(y_true, y_pred):
    # Ensure inputs are numpy arrays
    y_true = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()
    
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    # Calculate Huber Loss
    delta = 1.0
    abs_diff = np.abs(y_true - y_pred)
    huber_loss = np.mean(np.where(abs_diff < delta, 0.5 * abs_diff**2, delta * abs_diff - 0.5 * delta))
    
    # Calculate Log-Cosh Loss
    log_cosh_loss = np.mean(np.log(np.cosh(y_pred - y_true)))
    
    return mse, mae, r2, huber_loss, log_cosh_loss


In [23]:
print("Shape of y_test:", y_test.shape)
print("Shape of y_pred_rf:", y_pred_rf.shape)
print("Shape of y_pred_xgb:", y_pred_xgb.shape)
print("Shape of y_pred_lstm:", y_pred_lstm.shape)
print("Shape of y_pred_gru:", y_pred_gru.shape)
print("Shape of y_pred_rnn:", y_pred_rnn.shape)


Shape of y_test: (101,)
Shape of y_pred_rf: (101,)
Shape of y_pred_xgb: (101,)
Shape of y_pred_lstm: (101, 1)
Shape of y_pred_gru: (101, 1)
Shape of y_pred_rnn: (101, 1)


In [24]:
# Example calculations
metrics_rf = calculate_metrics(y_test, y_pred_rf)
metrics_xgb = calculate_metrics(y_test, y_pred_xgb)
metrics_lstm = calculate_metrics(y_test, y_pred_lstm)
metrics_gru = calculate_metrics(y_test, y_pred_gru)
metrics_rnn = calculate_metrics(y_test, y_pred_rnn)

print("Metrics for RF:", metrics_rf)
print("Metrics for XGB:", metrics_xgb)
print("Metrics for LSTM:", metrics_lstm)
print("Metrics for GRU:", metrics_gru)
print("Metrics for RNN:", metrics_rnn)


Metrics for RF: (38647.0866610727, 113.00433930390295, 0.9772798824772148, 112.50433930390295, inf)
Metrics for XGB: (35335.17861676153, 114.72583152903186, 0.9792269099634341, 114.22583152903186, inf)
Metrics for LSTM: (2751828.3278229544, 1025.0956812754716, -0.6177639354544733, 1024.5984778679233, inf)
Metrics for GRU: (2752307.045308873, 1025.3292521675037, -0.6180453672124651, 1024.8305803168726, inf)
Metrics for RNN: (2752556.3868593685, 1025.4507036492378, -0.6181919518536374, 1024.9514823197217, inf)


C:\Users\goeld\AppData\Local\Temp\ipykernel_44996\1805134145.py:18: RuntimeWarning: overflow encountered in cosh
  log_cosh_loss = np.mean(np.log(np.cosh(y_pred - y_true)))
C:\Users\goeld\AppData\Local\Temp\ipykernel_44996\1805134145.py:18: RuntimeWarning: overflow encountered in cosh
  log_cosh_loss = np.mean(np.log(np.cosh(y_pred - y_true)))
C:\Users\goeld\AppData\Local\Temp\ipykernel_44996\1805134145.py:18: RuntimeWarning: overflow encountered in cosh
  log_cosh_loss = np.mean(np.log(np.cosh(y_pred - y_true)))
C:\Users\goeld\AppData\Local\Temp\ipykernel_44996\1805134145.py:18: RuntimeWarning: overflow encountered in cosh
  log_cosh_loss = np.mean(np.log(np.cosh(y_pred - y_true)))
C:\Users\goeld\AppData\Local\Temp\ipykernel_44996\1805134145.py:18: RuntimeWarning: overflow encountered in cosh
  log_cosh_loss = np.mean(np.log(np.cosh(y_pred - y_true)))
